In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.43


In [2]:
#create workspace on Azure and connect the local notebook to workspace
ws = Workspace.get(name="Dian_ws", subscription_id='49cf6cbe-2987-4a45-9d6e-955ab6e10fde', resource_group='Dian')
print(ws.name, ws.location, ws.resource_group, ws.location, sep='\t')

Dian_ws	westeurope	Dian	westeurope


In [3]:
# Create the BlockBlockService that is used to call the Blob service for the storage account.
from azure.storage.blob import BlockBlobService
block_blob_service = BlockBlobService(account_name = 'dianws2583025325', account_key = 'EJbizu7oi2MT9v5ADB7Y/8+QZlZ7cVkcItnTPBTr8Xooa6iVzsV2ZVZIM/qTSRW14Xt1GhmkuK5OJaJKKDZ0ug==')

In [126]:
block_blob_service

### Methode2 :  Use data_folder to upload data files

In [151]:
df = pd.read_csv('./DATA-rti/RTI.csv',sep=",")
serie = pd.Series(df['FR0010172437'].values, index=pd.to_datetime(df['edvie_his_vl.db_val_liq']) )

X = serie
step_len = 2
step_forecast = 1
#### fill na###################
X = X.fillna(method='pad').dropna()

df,df_name =[], []
#### delay ###################
for i in range(step_len):
    df.append(X.shift(step_forecast + i))
    df_name.append('step_delay_{}'.format(i))

### day of week#################
df.append(pd.Series(X.index.dayofweek, index=X.index))
df_name.append('day_of_week')

df=pd.concat(df, axis=1)
df.columns=df_name
        
###moving diff################
df['diff_mov'] = df['step_delay_0']-df['step_delay_1']

y_train = X[(step_len+step_forecast-1):]

X_train = df[(step_len+step_forecast-1):]

y_train.to_csv('./DATA-rti/target_train.csv')
X_train.to_csv('./DATA-rti/x_train.csv')


C:\Users\dian.kang\AppData\Local\Continuum\anaconda3\envs\tfenv\lib\site-packages\ipykernel_launcher.py:30: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [134]:
import os
data_folder = os.path.join(os.getcwd(), 'DATA-rti')
os.makedirs(data_folder, exist_ok=True)


ds = ws.get_default_datastore()
print(ds.datastore_type, ds.account_name, ds.container_name)

ds.upload(src_dir=data_folder, target_path='rti_data', overwrite=True, show_progress=True)

AzureBlob dianws2583025325 azureml-blobstore-69868bc8-70ba-42ae-b703-53ac7320976a
Uploading C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\.ipynb_checkpoints\RTI-checkpoint.csv
Uploading C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\RTI.csv
Uploading C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\target_train.csv
Uploading C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\x_train.csv
Uploaded C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\target_train.csv, 1 files out of an estimated total of 4
Uploaded C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\x_train.csv, 2 files out of an estimated total of 4
Uploaded C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\.ipynb_checkpoints\RTI-checkpoint.csv, 3 files out of an estimated total of 4
Uploaded C:\Users\dian.kang\Desktop\Formation\ts\DATA-rti\RTI.csv, 4 files out of an estimated total of 4


$AZUREML_DATAREFERENCE_63273ea5b044474790f73d7ba3518ce1

### Train on a remote cluster

In [152]:
#Create a directory to deliver the necessary code from your computer to the remote resource.
import os
script_folder = os.path.join(os.getcwd(), "sklearn-rti")
os.makedirs(script_folder, exist_ok=True)

In [167]:
%%writefile $script_folder/train.py

import argparse
import os
import joblib
from azureml.core import Run

from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

# let user feed in 2 parameters, the location of the data files (from datastore), and the regularization rate of the logistic regression model
parser = argparse.ArgumentParser()
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder mounting point')
args = parser.parse_args()

data_folder = args.data_folder
print('Data folder:', data_folder)
    
# load train and test set into numpy arrays
y_train = pd.Series.from_csv(os.path.join(data_folder, 'target_train.csv'),sep=",")
X_train = pd.read_csv(os.path.join(data_folder, 'x_train.csv'),sep=",", index_col=0)

# get hold of the current run
run = Run.get_context()

model = LinearRegression()
model.fit(X_train, y_train)

# calculate accuracy on the prediction
accuracy = model.score(X_train, y_train)

run.log('accuracy', np.float(accuracy))

os.makedirs('outputs', exist_ok=True)
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/sklearn_rti_model_v2.pkl')

Overwriting C:\Users\dian.kang\Desktop\Formation\ts\sklearn-rti/train.py


In [168]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [169]:
from azureml.core import Experiment
experiment_name = 'my_experiment_rti_v2'

exp = Experiment(workspace=ws, name=experiment_name)


In [170]:
from azureml.train.estimator import Estimator

script_params = {
    '--data-folder': ds.path('rti_data').as_mount()  
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=cpu_cluster,
                entry_script='train.py',
                conda_packages=['pandas','scikit-learn'])



In [171]:
run = exp.submit(config=est)
run

Experiment,Id,Type,Status,Details Page,Docs Page
my_experiment_rti_v2,my_experiment_rti_v2_1561313025_d10df957,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [208]:
# register model 
model = run.register_model(model_name='sklearn_rti_v3', model_path='outputs/sklearn_rti_model_v2.pkl')
print(model.name, model.id, model.version, sep='\t')

sklearn_rti_v3	sklearn_rti_v3:2	2


### Model Deploiment

In [174]:
#Retrieve the model 
from azureml.core import Workspace
from azureml.core.model import Model
import os 

ws = Workspace.get(name="Dian_ws", subscription_id='49cf6cbe-2987-4a45-9d6e-955ab6e10fde', resource_group='Dian')
model= Model(ws, 'sklearn_rti_v2',version=1)

model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), "sklearn_rti_model_v2.pkl")

os.stat(file_path)

os.stat_result(st_mode=33206, st_ino=18014398509933042, st_dev=2824130541, st_nlink=1, st_uid=0, st_gid=0, st_size=622, st_atime=1561313498, st_mtime=1561313498, st_ctime=1561313498)

In [175]:
# all the packages must be installed for test data
#from jours_feries_france.compute import JoursFeries
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

import pickle
import joblib

model = joblib.load(os.path.join(os.getcwd(), 'sklearn_rti_model_v2.pkl'))

C:\Users\dian.kang\AppData\Local\Continuum\anaconda3\envs\tfenv\lib\site-packages\sklearn\base.py:253: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.2 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [176]:
model.predict(X_train)

array([1602.58955912, 1606.95680895, 1609.78598307, 1616.3432072 ,
       1615.18012722, 1618.87403936, 1620.38800969, 1616.66855248,
       1613.64281653, 1614.59298953, 1614.2553854 , 1612.93239594,
       1615.05643547, 1614.89239778, 1615.62147728, 1617.28108172,
       1616.78418876, 1618.29686738, 1617.87767572, 1616.83842181,
       1612.23102169, 1617.33357518, 1616.48332472, 1615.93933791,
       1608.46239199, 1607.10924614, 1605.52512761, 1599.16601768,
       1599.73063495, 1587.96882508, 1583.28899946, 1588.8656203 ,
       1580.45874464, 1574.54235757, 1575.14611299, 1580.39841188,
       1575.35968514, 1580.58452044, 1584.96944995, 1591.82226524,
       1590.59266737, 1591.60686733, 1592.74866325, 1591.73804959,
       1589.50245711, 1593.86163397, 1593.07072227, 1598.26041413,
       1595.75031682, 1592.95816185, 1587.44543819, 1575.08630978,
       1576.48056048, 1578.46054762, 1579.41401634, 1580.71950854,
       1586.47918658, 1590.18625752, 1589.34462517, 1592.35312

In [209]:
%%writefile score.py

import json
import numpy as np
import os
import pickle
import joblib
import argparse
import pandas as pd

from sklearn.linear_model import LinearRegression
from azureml.core.model import Model

# retrieve the path to the model file using the model name
def init():
    
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sklearn_rti_v3',version=2)
    model = joblib.load(model_path)
    
def run(raw_data):
    
    # load train and test set into numpy arrays
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()


Overwriting score.py


In [210]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_conda_package("pandas")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [211]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "rti",  "method" : "sklearn"}, 
                                               description='Predict rti with sklearn')

In [212]:
%%time

from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='sklearn-image-svc1',
                                       deployment_config=aciconfig,
                                       models=[model],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Running...................................
Succeeded
Image creation operation finished for image sklearn-image-svc1:1, operation "Succeeded"
Creating service
Running...............
SucceededACI service creation operation finished, operation "Succeeded"
Wall time: 4min 32s
